In [14]:
import pandas as pd
import numpy as np
import os
from scipy.integrate import cumulative_trapezoid

# ==========================================
# 1. 사용자 설정 (Path 및 파라미터)
# ==========================================
folder_path = r"G:\다른 컴퓨터\KHU PC\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\1. 실험자료\5. Revision experiment\260111\Figure 1f\Current Conv 8cm_50uAV"

output_filename = "00Results_Summary.csv"

fraction_percent = 10  # baseline 추정 비율 (%)
Current_scale = 50     # μA/V 스케일링 계수


In [15]:
# ==========================================
# 2. 전하 적분 함수 정의
# ==========================================
def segment_and_integrate(current, time, df_full, threshold=0.01, scale_factor=1e3):
    if len(time) < 1:
        return 0.0
    total_charge = 0.0
    group_start = 0
    time_diffs = np.diff(time)
    
    for i, dt in enumerate(time_diffs):
        if dt > threshold:
            segment_t = time[group_start:i+1]
            segment_i = current[group_start:i+1]
            if len(segment_t) > 1:
                total_charge += scale_factor * cumulative_trapezoid(segment_i, segment_t, initial=0)[-1]
            elif len(segment_t) == 1:
                dt_est = 1e-6
                t0 = segment_t[0]
                if t0 in df_full["Time"].values:
                    idx = df_full.index[df_full["Time"] == t0].tolist()[0]
                    if 0 < idx < len(df_full) - 1:
                        dt_est = (df_full.iloc[idx + 1]["Time"] - df_full.iloc[idx - 1]["Time"]) / 2
                total_charge += scale_factor * segment_i[0] * dt_est
            group_start = i + 1
            
    if group_start < len(time):
        segment_t = time[group_start:]
        segment_i = current[group_start:]
        if len(segment_t) > 1:
            total_charge += scale_factor * cumulative_trapezoid(segment_i, segment_t, initial=0)[-1]
        elif len(segment_t) == 1:
            dt_est = 1e-6
            t0 = segment_t[0]
            if t0 in df_full["Time"].values:
                idx = df_full.index[df_full["Time"] == t0].tolist()[0]
                if 0 < idx < len(df_full) - 1:
                    dt_est = (df_full.iloc[idx + 1]["Time"] - df_full.iloc[idx - 1]["Time"]) / 2
            total_charge += scale_factor * segment_i[0] * dt_est
            
    return total_charge

# ==========================================
# 3. 폴더 순회 및 데이터 처리 (정렬 추가)
# ==========================================
results_list = []

# 파일 이름을 순서대로 정렬 (sorted 사용)
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv') and f != output_filename])

print(f"총 {len(file_list)}개의 파일을 순서대로 처리합니다.")

for filename in file_list:
    full_path = os.path.join(folder_path, filename)
    
    try:
        df = pd.read_csv(full_path, header=None, skiprows=30)
        df.columns = ['Time', 'Current']
        df["Time"] = pd.to_numeric(df["Time"], errors='coerce')
        df["Current"] = pd.to_numeric(df["Current"], errors='coerce')
        df = df.dropna(subset=['Time', 'Current'])

        # Baseline Correction
        current_raw = df['Current'].values
        n_total = len(current_raw)
        n_fraction = int(n_total * (fraction_percent / 100))
        
        baseline_offset = np.mean(np.concatenate([current_raw[:n_fraction], current_raw[-n_fraction:]]))
        baseline_offset = 0.0 if np.isnan(baseline_offset) else baseline_offset
        
        # 보정된 전류 (스케일 적용)
        df['Cor_current'] = (df['Current'] - baseline_offset) * Current_scale
        
        # 1. 전류 최대/최소값
        max_current = df['Cor_current'].max()
        min_current = df['Cor_current'].min()
        
        # 2. 전하 적분 (양수/음수 분리)
        p_df = df[df["Cor_current"] > 0]
        n_df = df[df["Cor_current"] < 0]
        
        p_charge = segment_and_integrate(p_df["Cor_current"].values, p_df["Time"].values, df)
        n_charge = segment_and_integrate(n_df["Cor_current"].values, n_df["Time"].values, df)
        
        # 결과 정리
        results_list.append({
            "File Name": filename,
            "Max Current (uA)": max_current,
            "Min Current (uA)": min_current,
            "Max Charge (nC)": p_charge,
            "Min Charge (nC)": n_charge,
            "Offset (V)": baseline_offset
        })
        print(f"완료: {filename}")
        
    except Exception as e:
        print(f"오류 ({filename}): {e}")

# ==========================================
# 4. 결과 저장 (소수점 처리 추가)
# ==========================================
summary_df = pd.DataFrame(results_list)

# 모든 수치형 데이터를 소수점 3자리까지 반올림
summary_df = summary_df.round(3)

output_path = os.path.join(folder_path, output_filename)
summary_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("-" * 30)
print(f"모든 작업이 완료되었습니다.")
print(f"결과 파일 저장 경로: {output_path}")
print(summary_df)

총 500개의 파일을 순서대로 처리합니다.
완료: tek0000ALL.csv
완료: tek0001ALL.csv
완료: tek0002ALL.csv
완료: tek0003ALL.csv
완료: tek0004ALL.csv
완료: tek0005ALL.csv
완료: tek0006ALL.csv
완료: tek0007ALL.csv
완료: tek0008ALL.csv
완료: tek0009ALL.csv
완료: tek0010ALL.csv
완료: tek0011ALL.csv
완료: tek0012ALL.csv
완료: tek0013ALL.csv
완료: tek0014ALL.csv
완료: tek0015ALL.csv
완료: tek0016ALL.csv
완료: tek0017ALL.csv
완료: tek0018ALL.csv
완료: tek0019ALL.csv
완료: tek0020ALL.csv
완료: tek0021ALL.csv
완료: tek0022ALL.csv
완료: tek0023ALL.csv
완료: tek0024ALL.csv
완료: tek0025ALL.csv
완료: tek0026ALL.csv
완료: tek0027ALL.csv
완료: tek0028ALL.csv
완료: tek0029ALL.csv
완료: tek0030ALL.csv
완료: tek0031ALL.csv
완료: tek0032ALL.csv
완료: tek0033ALL.csv
완료: tek0034ALL.csv
완료: tek0035ALL.csv
완료: tek0036ALL.csv
완료: tek0037ALL.csv
완료: tek0038ALL.csv
완료: tek0039ALL.csv
완료: tek0040ALL.csv
완료: tek0041ALL.csv
완료: tek0042ALL.csv
완료: tek0043ALL.csv
완료: tek0044ALL.csv
완료: tek0045ALL.csv
완료: tek0046ALL.csv
완료: tek0047ALL.csv
완료: tek0048ALL.csv
완료: tek0049ALL.csv
완료: tek0050ALL.csv
완료: tek